<a href="https://colab.research.google.com/github/advcloud/colabtest/blob/main/azure1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Github/test3

/content/drive/MyDrive/Github/test3


In [2]:
%pip install azure-cognitiveservices-vision-customvision
%pip install pylabel

#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#Import other libraries used in this notebook 
import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer

     |████████████████████████████████| 62 kB 684 kB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 41 kB 645 kB/s 
     |████████████████████████████████| 338 kB 11.1 MB/s 


In [4]:
# Replace with your Azure endpoint and subscription keys.
ENDPOINT = ""
training_key = ""
prediction_key = ""
prediction_resource_id = ""

In [5]:
#Initialize objects used by Azure Congitive vision
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [6]:
#Create a new project
publish_iteration_name = "detectModel"
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
project = trainer.create_project("PyLabel Sample Dataset", domain_id=obj_detection_domain.id)
#If you browse to https://www.customvision.ai/ you should see a new project called "PyLabel Sample Dataset"

In [7]:
%%capture
os.makedirs("data/", exist_ok=True)
!wget "https://github.com/pylabel-project/datasets_models/blob/main/squirrelsandnuts/squirrelsandnuts_train.zip?raw=true" -O data/squirrelsandnuts_train.zip
with zipfile.ZipFile("data/squirrelsandnuts_train.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")

In [8]:
#Import annotations as a PyLabel dataset
dataset = importer.ImportYoloV5(path="data/squirrelsandnuts_train/labels/train",
        path_to_images="../../images/train", 
        img_ext="jpeg",
        cat_names=['Squirrel','Nut']
    )
dataset.df.head(3)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../../images/train,2021-07-03T06-30-10-frame_0001.jpeg,,0,960,540,3,,255.024,170.991,...,,,,,,0,Squirrel,,,1
1,../../images/train,2021-07-03T06-47-39-frame_0004.jpeg,,1,960,540,3,,650.016,447.984,...,,,,,,1,Nut,,,1
2,../../images/train,2021-07-03T06-47-39-frame_0004.jpeg,,1,960,540,3,,690.480,422.010,...,,,,,,1,Nut,,,1


In [9]:
print(dataset.analyze.classes)
#Create a tag for each class and store then in a dict where the class name is the key
tags = {}
for class_name in dataset.analyze.classes:
    tag = trainer.create_tag(project.id, class_name)
    tags[class_name] = tag

['Squirrel', 'Nut']


In [10]:
#Iterate the rows for each image in the dataframe
for img_filename, img_df in dataset.df.groupby('img_filename'):
    img_path = str(PurePath(dataset.path_to_annotations, str(img_df.iloc[0].img_folder), img_filename))
    assert exists(img_path), f"File does not exist: {img_path}"

    #Create a region object for each bounding box in the dataset 
    regions = []
    for index, row in img_df.iterrows():

        #Normalize the boundings box coordinates between 0 and 1
        x = Decimal(row.ann_bbox_xmin / row.img_width).min(1)
        y = Decimal(row.ann_bbox_ymin / row.img_height).min(1)
        w = Decimal(row.ann_bbox_width / row.img_width).min(1-x)
        h = Decimal(row.ann_bbox_height / row.img_height).min(1-y)
        
        regions.append(Region(
                tag_id=tags[row.cat_name].id, 
                left=x,
                top=y,
                width=w,
                height=h
            )
        )

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)]

    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(
            project.id, 
            ImageFileCreateBatch(images=image_and_annotations)
        )
    
    #If upload is not successful, print details about that image for debugging 
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            print(regions)

#This will take a few minutes 
print("Upload complete")

Upload complete


In [11]:
import os, time, uuid

In [12]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


In [13]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!
